# Regression Models For Predicting Price
___

## Why Regression Matters

Predicting Airbnb prices using regression models offers several advantages for both hosts and guests alike. By leveraging various predictors such as host response metrics, accommodation attributes, and review scores, these models can provide valuable insights into pricing strategies and help optimize the booking experience.

## Objectives

Utilize regression models to inform hosts' pricing strategies, enabling them to base decisions on empirical data rather than intuition alone. Enhance the guest experience by providing transparent and predictable pricing, guided by regression models that consider various factors impacting accommodation quality and value. Deliver personalized recommendations to hosts regarding optimal pricing strategies and to guests regarding relevant listings, leveraging regression models to analyze historical data and user preferences.

## Approach

In this project, we will employ three different regression models to predict Airbnb prices: Simple Linear Regression (SLR), Random Forest Regression, and Neural Network Regression. Each model offers unique advantages.

1. **Simple Linear Regression (SLR)**:
   SLR provides a straightforward approach to modeling the relationship between the predictor variables and the target variable (price). It's particularly useful for identifying linear relationships and understanding the individual impact of each predictor on the target variable. We'll use SLR to establish a baseline performance and interpret the coefficients of the predictors.

2. **Random Forest Regression**:
   Random Forest Regression is an ensemble learning technique that combines multiple decision trees to improve predictive accuracy and handle nonlinear relationships and interactions between predictors. It's robust to overfitting and can handle a large number of input variables with ease. We'll leverage Random Forest Regression to capture complex interactions between predictors and enhance prediction accuracy.

3. **Neural Network Regression**:
   Neural Network Regression, specifically deep learning architectures, can capture intricate patterns and relationships within the data, allowing for highly flexible modeling without relying on explicit feature engineering. Neural networks excel at learning nonlinear relationships and can adapt to complex datasets with high dimensionality. We'll utilize Neural Network Regression to capture nonlinearities and interactions that may not be captured effectively by traditional regression models.

For evaluation, we'll use Mean Squared Error (MSE) as the scoring metric for all three models. MSE provides a measure of the average squared difference between the predicted and actual prices, allowing us to assess the accuracy of the models' predictions.


# Simple Linear Regression (SLR)

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [17]:
df = pd.read_csv('../data/03_data.csv', index_col = False)

In [18]:
df.head()

,Unnamed: 0,price,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,accommodates,beds,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,0,83.0,within a day,100.0,17.0,0.0,4,2.0,4.63,4.67,4.52,4.64,4.69,4.92,4.62
1,2,116.0,within a few hours,100.0,99.0,0.0,4,2.0,4.69,4.73,4.73,4.78,4.79,4.72,4.62
2,3,70.0,within a few hours,100.0,47.0,0.0,1,1.0,4.68,5.00,4.85,4.70,4.85,4.60,4.45
3,4,100.0,within a day,100.0,100.0,0.0,2,2.0,4.40,4.07,4.47,4.80,4.73,4.87,4.00
4,8,25.0,within an hour,100.0,96.0,1.0,2,1.0,4.77,4.82,4.71,4.89,4.92,4.87,4.65


In [19]:
# All of the columns in this dataframe are ready to go except host_response_time which needs to be ordinal encoded
df['host_response_time'].value_counts()

host_response_time
within an hour        4043
within a few hours    1092
within a day           906
a few days or more     161
Name: count, dtype: int64

In [20]:
ordinal_encoding = {'within an hour': 0, 'within a few hours': 1, 'within a day': 2, 'a few days or more': 3}

# Map the values in the 'host_response_time' column to their ordinal encoding
df['host_response_time_encoded'] = df['host_response_time'].map(ordinal_encoding)

In [21]:
X = df.drop(columns=['price', 'host_response_time'])
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define parameter grid for Linear Regression
param_grid_lr = {
    'fit_intercept': [True, False],
    'positive': [True, False]
}

# Initialize Linear Regression model
lr_model = LinearRegression()

# Perform Grid Search with 5-fold Cross-Validation for Linear Regression
grid_search_lr = GridSearchCV(estimator=lr_model, param_grid=param_grid_lr, cv=5, scoring='r2')
grid_search_lr.fit(X_train_scaled, y_train)

# Get best Linear Regression model from grid search
best_lr_model = grid_search_lr.best_estimator_

# Predict on the scaled test set using the best Linear Regression model
y_pred_lr = best_lr_model.predict(X_test_scaled)

# Calculate mean squared error
mse_lr = mean_squared_error(y_test, y_pred_lr)
print("Mean Squared Error (Linear Regression):", mse_lr)

# Calculate R-squared score
r2_lr = r2_score(y_test, y_pred_lr)
print("R-squared (Linear Regression):", r2_lr)

Mean Squared Error (Linear Regression): 9325.434233710608
R-squared (Linear Regression): 0.21078797725892573


An R-squared score of 0.21 indicates that approximately 21% of the variance in Airbnb prices can be explained by the predictors included in our model. This suggests that while our model captures some of the factors influencing Airbnb prices, there are still other many unaccounted variables contributing to the variability in prices.

# Random Forest Regression (RFR)

In [22]:
# Initialize Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Define hyperparameters grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search with 5-fold Cross-Validation for Random Forest
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, scoring='r2')
grid_search_rf.fit(X_train_scaled, y_train)

# Get best Random Forest model from grid search
best_rf_model = grid_search_rf.best_estimator_

# Initialize Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(random_state=42)

# Define hyperparameters grid for Gradient Boosting
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

# Perform Grid Search with 5-fold Cross-Validation for Gradient Boosting
grid_search_gb = GridSearchCV(estimator=gb_model, param_grid=param_grid_gb, cv=5, scoring='r2')
grid_search_gb.fit(X_train_scaled, y_train)

# Get best Gradient Boosting model from grid search
best_gb_model = grid_search_gb.best_estimator_

# Predict on the scaled test set using the best Random Forest model
y_pred_rf = best_rf_model.predict(X_test_scaled)

# Calculate R-squared score for Random Forest
r2_rf = r2_score(y_test, y_pred_rf)
print("R-squared (Random Forest):", r2_rf)

# Predict on the scaled test set using the best Gradient Boosting model
y_pred_gb = best_gb_model.predict(X_test_scaled)

# Calculate R-squared score for Gradient Boosting
r2_gb = r2_score(y_test, y_pred_gb)
print("R-squared (Gradient Boosting):", r2_gb)

R-squared (Random Forest): 0.21216177264397096
R-squared (Gradient Boosting): 0.14607777287241186


Despite gradient boosting and gridsearching and using best hyperparameters, we end up with only a R-squared scores of 0.21 and 0.15 again. Just as before, we can determine there are other predictors which would predict price better than the ones we have.

# Neural Network Regression (NNR)